<a href="https://colab.research.google.com/github/MOOwuttichai/BSC_DPDM2023/blob/main/%E0%B8%BABERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertModel, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd

In [2]:
df = pd.read_csv("/content/Data_model.csv")
labels = []
for sentiment in df['label']:
  if sentiment == "เล่าประสบการณ์ (tell experience)":
    labels.append(1) #เล่าประสบการณ์ (tell experience)
  elif sentiment == "คำถาม (Question)":
    labels.append(2) #ไม่มีประโยชน์/ไม่สำคัญ (useless/unimportant)
  else:
    labels.append(0)
df['label_num'] = labels

In [3]:
train_df_E, test_df_E = train_test_split(df, test_size=0.3, random_state=42) # 70/30 split, random_state for reproducibility
print("Training set size:", len(train_df_E))
print("Testing set size:", len(test_df_E))

Training set size: 2168
Testing set size: 930


In [4]:
# prompt: ช่วยเเก้ไข Imbalance ของข้อมูล df โดยที่ x คือคอลัม comments, y คือคอลัม label โดยใช้ imblearn

from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

# Separate features (X) and target (y)
X = train_df_E['comments'].values.reshape(-1, 1)  # Reshape for imblearn
y = train_df_E['label_num']

# Choose either oversampling or undersampling
# Oversampling:
oversampler = RandomOverSampler(random_state=42)
X_resampled, y_resampled = oversampler.fit_resample(X, y)

# # Undersampling:
# undersampler = RandomUnderSampler(random_state=42)
# X_resampled, y_resampled = undersampler.fit_resample(X, y)

# Create a new DataFrame with the resampled data
df_resampled = pd.DataFrame({'comments': X_resampled.flatten(), 'label': y_resampled})

# Now use df_resampled instead of df for training
texts = df_resampled['comments'].tolist()
labels = df_resampled['label'].tolist()
print(len(texts))
print(len(labels))

4701
4701


In [5]:
df_resampled

,comments,label
0,คุณป้ามะเร็งลำไส้ใหญ่ เริ่มผิดปกติมีเลือดซึมออ...,1
1,Phonpin Mongkon,0
2,ขอแชร์นะครับ ได้รับความรู้ดีมากๆครับ,0
3,Sopa Jadurai,0
4,Somkiat Suksongkram,0
...,...,...
4696,มาหาข้อมูลว่าเสี่ยงมะเร็งตับไหม สูงทุกปีตรวจล่...,2
4697,สอบถามค่ะเรามีอาการเหมือนท้องผูกเวลาถ่ายท้องยา...,2
4698,ตรวจยังไงให้เจอเป๊ะๆ​ ไปทีไรปกติทุกที​ สมัยก่อ...,2
4699,วัดน้ำตาลปลายนิ้ว 143 ถือว่าสูงมากใช่ไหมฮะ ต้อ...,2


In [6]:
# train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=0.3, random_state=42)

In [7]:
# prompt: ช่วยเเปลงข้อมูลใน list จาก str เป็น int

# Assuming 'labels' is a list of strings representing integers


In [8]:
set(labels)

{0, 1, 2}

In [9]:
class TextClassificationDataset(Dataset):
  def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
  def __len__(self):
      return len(self.texts)
  def __getitem__(self, idx):
      text = self.texts[idx]
      label = self.labels[idx]
      encoding = self.tokenizer(text, return_tensors='pt', max_length=self.max_length, padding='max_length', truncation=True)
      return {'input_ids': encoding['input_ids'].flatten(), 'attention_mask': encoding['attention_mask'].flatten(), 'label': torch.tensor(label)}

In [10]:
class BERTClassifier(nn.Module):
  def __init__(self, bert_model_name, num_classes):
        super(BERTClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_name)
        self.dropout = nn.Dropout(0.1)
        self.fc = nn.Linear(self.bert.config.hidden_size, num_classes)
  def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        x = self.dropout(pooled_output)
        logits = self.fc(x)
        return logits

In [11]:
def train(model, data_loader, optimizer, scheduler, device):
    model.train()
    for batch in data_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = nn.CrossEntropyLoss()(outputs, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()

In [12]:
def evaluate(model, data_loader, device):
    model.eval()
    predictions = []
    actual_labels = []
    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs, dim=1)
            predictions.extend(preds.cpu().tolist())
            actual_labels.extend(labels.cpu().tolist())
    return accuracy_score(actual_labels, predictions), classification_report(actual_labels, predictions)

In [13]:
def predict_sentiment(text, model, tokenizer, device, max_length=128):
    model.eval()
    encoding = tokenizer(text, return_tensors='pt', max_length=max_length, padding='max_length', truncation=True)
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        _, preds = torch.max(outputs, dim=1)
    if preds.item() == 1:
      L = "เล่าประสบการณ์ (tell experience)"
    elif preds.item() == 2:
      L = "คำถาม (Question)"
    else:
      L = "ไม่มีประโยชน์/ไม่สำคัญ (useless/unimportant)"
    return L

In [14]:
# Set up parameters
bert_model_name = 'airesearch/wangchanberta-base-att-spm-uncased'
num_classes = 3
max_length = 512
batch_size = 16
num_epochs = 10
learning_rate = 2e-5

In [15]:
train_texts=texts
train_labels=labels
val_texts = test_df_E['comments'].tolist()
val_labels = test_df_E['label_num'].tolist()

In [16]:
train_texts

['คุณป้ามะเร็งลำไส้ใหญ่ เริ่มผิดปกติมีเลือดซึมออกมาทางทวารหนัก แต่ไปคิดว่าคงเป็นจากการถ่ายท้องเลยเป็นแผลหรือริดสีดวง ต่อมาเลือดไหลเยอะขึ้นตัวเริ่มซีดเหลืองเลยไปตรวจ เจอมะเร็งลำไส้ใหญ่ระยะ 3 ลามไปต่อน้ำเหลือง เริ่มฉายแสงคุณป้าเริ่มซูบผอมเบื่ออาหาร มีคนเอายาลูกกลอนมาให้ป้าทานหลังทานอาการดีขึ้นทานอาหารได้ ผิวจากเหลืองเริ่มชมพู มีเนื้อหนังขึ้น แกเลยทานต่อเรื่อยๆ ห้ามแล้วก็ไม่เชื่อ',
 'Phonpin Mongkon',
 'ขอแชร์นะครับ ได้รับความรู้ดีมากๆครับ',
 'Sopa Jadurai',
 'Somkiat Suksongkram',
 'Chatchai Siliputthaivun',
 'ดิฉันเป็นมะเร็งเต้านมระยะที่ 1 ขนาด 0.8 cm ดิฉันเป็นคนที่จะตรวจคลำเต้านมหลังอาบน้ำเกือบทุกวันตั้งแต่อายุ 35 ปี พอมีอะไรผิดปกติก็รีบไปหาหมอ คุณหมอบอกว่า คนไข้คลำเจอได้ยังไง ขนาดหมอยังคลำไม่เจอเลย ผลตรวจก็แจ๊คพอตค่ะ เมมโมแกรม อัลตราซาวน์ ตรวจชิ้นเนื้อ สร... ดูเพิ่มเติม',
 'กินเก่งจัว',
 'ก่อนหน้านี้ รู้สึกตัวเอง เพลียง่าย แล้วมาคลำเจอก้อนที่เต้านมค่ะ',
 'ปี63เป็นมะเร็งรังไข่ค่ะระยะ1C อาการ ท้องโต ตอนแรกคิดว่าหน้าท้อง มีไข้ต่ำๆตอนหัวค่ำตลอดแทบทุกวัน ท้องแข็ง ปวดบิด เลยหาหมอถึงตรวจเจอค

In [17]:
print(len(train_texts))
print(len(train_labels))

4701
4701


In [18]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(bert_model_name)
train_dataset = TextClassificationDataset(train_texts, train_labels, tokenizer, max_length)
val_dataset = TextClassificationDataset(val_texts, val_labels, tokenizer, max_length)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [19]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BERTClassifier(bert_model_name, num_classes).to(device)

You are using a model of type camembert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of BertModel were not initialized from the model checkpoint at airesearch/wangchanberta-base-att-spm-uncased and are newly initialized: ['embeddings.LayerNorm.bias', 'embeddings.LayerNorm.weight', 'embeddings.position_embeddings.weight', 'embeddings.token_type_embeddings.weight', 'embeddings.word_embeddings.weight', 'encoder.layer.0.attention.output.LayerNorm.bias', 'encoder.layer.0.attention.output.LayerNorm.weight', 'encoder.layer.0.attention.output.dense.bias', 'encoder.layer.0.attention.output.dense.weight', 'encoder.layer.0.attention.self.key.bias', 'encoder.layer.0.attention.self.key.weight', 'encoder.layer.0.attention.self.query.bias', 'encoder.layer.0.attention.self.query.weight', 'encoder.layer.0.attention.self.value.bias', 'encoder.layer.0.attention.self.value.weight', 'encoder.layer.0.intermediate.dense.bias', 

In [20]:
optimizer = AdamW(model.parameters(), lr=learning_rate)
total_steps = len(train_dataloader) * num_epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
for epoch in range(num_epochs):
  print(f"Epoch {epoch + 1}/{num_epochs}")
  train(model, train_dataloader, optimizer, scheduler, device)
  accuracy, report = evaluate(model, val_dataloader, device)
  print(f"Validation Accuracy: {accuracy:.4f}")
  print(report)

Epoch 1/10


In [73]:
torch.save(model.state_dict(), "bert_classifier.pth")

KeyboardInterrupt: 

In [ ]:
# prompt: เรียกใช้ bert_classifier.pth

import os
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertModel, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from transformers import AutoTokenizer
bert_model_name = 'airesearch/wangchanberta-base-att-spm-uncased'
num_classes = 3
max_length = 200
batch_size = 16
num_epochs = 4
learning_rate = 2e-5
# Load the saved model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
bert_model_name = 'airesearch/wangchanberta-base-att-spm-uncased'
num_classes = 3
model = BERTClassifier(bert_model_name, num_classes).to(device)
model.load_state_dict(torch.load("bert_classifier.pth", map_location=device))
tokenizer = AutoTokenizer.from_pretrained(bert_model_name)

# Example usage:
# text_to_predict = "This is an example text." # Replace with your input
# predicted_sentiment = predict_sentiment(text_to_predict, model, tokenizer, device)
# print(f"Predicted sentiment: {predicted_sentiment}")

class TextClassificationDataset(Dataset):
  def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
  def __len__(self):
      return len(self.texts)
  def __getitem__(self, idx):
      text = self.texts[idx]
      label = self.labels[idx]
      encoding = self.tokenizer(text, return_tensors='pt', max_length=self.max_length, padding='max_length', truncation=True)
      return {'input_ids': encoding['input_ids'].flatten(), 'attention_mask': encoding['attention_mask'].flatten(), 'label': torch.tensor(label)}
class BERTClassifier(nn.Module):
  def __init__(self, bert_model_name, num_classes):
        super(BERTClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_name)
        self.dropout = nn.Dropout(0.1)
        self.fc = nn.Linear(self.bert.config.hidden_size, num_classes)
  def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        x = self.dropout(pooled_output)
        logits = self.fc(x)
        return logits
def predict_sentiment(text, model, tokenizer, device, max_length=128):
    model.eval()
    encoding = tokenizer(text, return_tensors='pt', max_length=max_length, padding='max_length', truncation=True)
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        _, preds = torch.max(outputs, dim=1)
    if preds.item() == 1:
      L = "เล่าประสบการณ์ (tell experience)"
    elif preds.item() == 2:
      L = "คำถาม (Question)"
    else:
      L = "ไม่มีประโยชน์/ไม่สำคัญ (useless/unimportant)"
    return L

df_use = pd.read_csv("/content/data_pre_chi2.csv")
text_use = df_use['comments'].tolist()
labels_use = []
for i in text_use:
  text_to_predict = i # Replace with your input
  predicted_sentiment = predict_sentiment(text_to_predict, model, tokenizer, device)
  labels_use.append(predicted_sentiment)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/546 [00:00<?, ?B/s]

You are using a model of type camembert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.


model.safetensors:   0%|          | 0.00/423M [00:00<?, ?B/s]

Some weights of BertModel were not initialized from the model checkpoint at airesearch/wangchanberta-base-att-spm-uncased and are newly initialized: ['embeddings.LayerNorm.bias', 'embeddings.LayerNorm.weight', 'embeddings.position_embeddings.weight', 'embeddings.token_type_embeddings.weight', 'embeddings.word_embeddings.weight', 'encoder.layer.0.attention.output.LayerNorm.bias', 'encoder.layer.0.attention.output.LayerNorm.weight', 'encoder.layer.0.attention.output.dense.bias', 'encoder.layer.0.attention.output.dense.weight', 'encoder.layer.0.attention.self.key.bias', 'encoder.layer.0.attention.self.key.weight', 'encoder.layer.0.attention.self.query.bias', 'encoder.layer.0.attention.self.query.weight', 'encoder.layer.0.attention.self.value.bias', 'encoder.layer.0.attention.self.value.weight', 'encoder.layer.0.intermediate.dense.bias', 'encoder.layer.0.intermediate.dense.weight', 'encoder.layer.0.output.LayerNorm.bias', 'encoder.layer.0.output.LayerNorm.weight', 'encoder.layer.0.output.d

tokenizer_config.json:   0%|          | 0.00/282 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/905k [00:00<?, ?B/s]

In [ ]:
df_use['label'] = labels_use

In [ ]:
LOL= df_use[['comments','label']]

In [ ]:
LOL.to_csv('data_label_chi2.csv', encoding='utf-8-sig',index=False)

In [ ]:
test_text = "ตอบยากนะคะ อาการของโรคต้องให้หมอเป็นคนประเมิน...ส่วนของแม่เรารักษาที่โรงพยาบาลรัฐบาล1เดือนกว่า มีอาการหนักกว่านี้ค่ะ จนถึงจุดที่คุณหมอเข้ามาคุยว่าจะรักษาต่อหรือจะกลับบ้าน รักษาแบบประคับประคอง เรารับรู้อาการของแม่ตลอดว่าแม่ไม่ไหวแล้ว เราเลือกพาแม่กลับบ้าน ท่านอยู่บ้านได้ 11 วันก็จากไปอย่างสงบ ระหว่างอยู่บ้านจะมีทีมรักษาแบบประคับประคองดูแลให้คำปรึกษาตลอดค่ะ"
sentiment = predict_sentiment(test_text, model, tokenizer, device)
print(test_text)
print(f"Predicted sentiment: {sentiment}")

ตอบยากนะคะ อาการของโรคต้องให้หมอเป็นคนประเมิน...ส่วนของแม่เรารักษาที่โรงพยาบาลรัฐบาล1เดือนกว่า มีอาการหนักกว่านี้ค่ะ จนถึงจุดที่คุณหมอเข้ามาคุยว่าจะรักษาต่อหรือจะกลับบ้าน รักษาแบบประคับประคอง เรารับรู้อาการของแม่ตลอดว่าแม่ไม่ไหวแล้ว เราเลือกพาแม่กลับบ้าน ท่านอยู่บ้านได้ 11 วันก็จากไปอย่างสงบ ระหว่างอยู่บ้านจะมีทีมรักษาแบบประคับประคองดูแลให้คำปรึกษาตลอดค่ะ
Predicted sentiment: เล่าประสบการณ์ (tell experience)
